Creative Commons CC BY 4.0 Lynd Bacon & Associates, Ltd. Not warranted to be suitable for any particular purpose. (You're on your own!)

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

<h1 align='center'>KMeans examples Assignment 2 v1</h1>

Assignment 2 asks you to do a cluster analysis using kmeans clustering.  kmeans is at partitioning method.  It starts out using an a priori specified number of clusters, and then attempts to assign data observations to clusters in order to optimize a criterion like maximizing the _homogeneity_ of clusters.  Clustering is an unsupervised learning method, meaning that there is no target variable to be predicted.  The task is to be able to discover whether there is structure in data in terms of similarities or differences between different data points.

In the following we'll train a kmeans clustering solution using two different methods provided in the _scikit-learn_ library.  In doing your assignment you need only use one of them.   We'll apply a metric that 

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.cluster import KMeans, MiniBatchKMeans
%matplotlib inline
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

## Example Data

Here we'll use a selection of the numeric data from the Ames dataset. 

Assuming that the pickle file is in the current working directory:

In [2]:
AmesSelDF=pd.read_pickle('amesSelDF.pickle')
AmesSelDF.columns

Index(['Lot_Frontage', 'Lot_Area', 'Mas_Vnr_Area', 'Bsmt_Unf_SF',
       'Total_Bsmt_SF', 'First_Flr_SF', 'Second_Flr_SF', 'Gr_Liv_Area',
       'Bedroom_AbvGr', 'Kitchen_AbvGr', 'TotRms_AbvGrd', 'Fireplaces',
       'Garage_Area', 'Wood_Deck_SF', 'Open_Porch_SF', 'Sale_Price'],
      dtype='object')

These variables seem "more or less" metric:

In [3]:
AmesSelDF.describe()

,Lot_Frontage,Lot_Area,Mas_Vnr_Area,Bsmt_Unf_SF,Total_Bsmt_SF,First_Flr_SF,Second_Flr_SF,Gr_Liv_Area,Bedroom_AbvGr,Kitchen_AbvGr,TotRms_AbvGrd,Fireplaces,Garage_Area,Wood_Deck_SF,Open_Porch_SF,Sale_Price
count,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000
mean,57.647782,10147.921843,101.096928,559.071672,1051.255631,1159.557679,335.455973,1499.690444,2.854266,1.044369,6.443003,0.599317,472.658362,93.751877,47.533447,180796.060068
std,33.499441,7880.017759,178.634545,439.540571,440.968018,391.890885,428.395715,505.508887,0.827731,0.214076,1.572964,0.647921,215.187196,126.361562,67.483400,79886.692357
min,0.000000,1300.000000,0.000000,0.000000,0.000000,334.000000,0.000000,334.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,12789.000000
25%,43.000000,7440.250000,0.000000,219.000000,793.000000,876.250000,0.000000,1126.000000,2.000000,1.000000,5.000000,0.000000,320.000000,0.000000,0.000000,129500.000000
50%,63.000000,9436.500000,0.000000,465.500000,990.000000,1084.000000,0.000000,1442.000000,3.000000,1.000000,6.000000,1.000000,480.000000,0.000000,27.000000,160000.000000
75%,78.000000,11555.250000,162.750000,801.750000,1301.500000,1384.000000,703.750000,1742.750000,3.000000,1.000000,7.000000,1.000000,576.000000,168.000000,70.000000,213500.000000
max,313.000000,215245.000000,1600.000000,2336.000000,6110.000000,5095.000000,2065.000000,5642.000000,8.000000,3.000000,15.000000,4.000000,1488.000000,1424.000000,742.000000,755000.000000


_Sale_Price_ is the target variable you'll be training emsembles to predict.  We'll set that aside here and cluster on the rest of the columns. Then we'll 

In [4]:
AmesSelDF.loc[:,~(AmesSelDF.columns.isin(['Sale_Price']))]

,Lot_Frontage,Lot_Area,Mas_Vnr_Area,Bsmt_Unf_SF,Total_Bsmt_SF,First_Flr_SF,Second_Flr_SF,Gr_Liv_Area,Bedroom_AbvGr,Kitchen_AbvGr,TotRms_AbvGrd,Fireplaces,Garage_Area,Wood_Deck_SF,Open_Porch_SF
0,141,31770,112,441,1080,1656,0,1656,3,1,7,2,528,210,62
1,80,11622,0,270,882,896,0,896,2,1,5,0,730,140,0
2,81,14267,108,406,1329,1329,0,1329,3,1,6,0,312,393,36
3,93,11160,0,1045,2110,2110,0,2110,3,1,8,2,522,0,0
4,74,13830,0,137,928,928,701,1629,3,1,6,1,482,212,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,37,7937,0,184,1003,1003,0,1003,3,1,6,0,588,120,0
2926,0,8885,0,239,864,902,0,902,2,1,5,0,484,164,0
2927,62,10441,0,575,912,970,0,970,3,1,6,0,0,80,32
2928,77,10010,0,195,1389,1389,0,1389,2,1,6,1,418,240,38


In [5]:
AmesClusDF=AmesSelDF.loc[:,~(AmesSelDF.columns.isin(['Sale_Price']))].astype('float32')
AmesClusDF.dtypes
X=AmesClusDF.to_numpy(copy=True)     # get the np array out of the DataFrame

### Example: 4 Cluster Solution: Kmeans

In [ ]:
km4=KMeans(n_clusters=4,random_state=33).fit(X)

In [ ]:
label=km4.predict(X)     # predicted cluster membership labels
pd.Series(label).value_counts()

## Clustering Metrics

Be sure to check the documentation to understand what these metrics are measuring.

In [ ]:
print(f'CH Score: {calinski_harabasz_score(X,label)}')
print(f'DB Score: {davies_bouldin_score(X,label)}')
print(f'Sihouette Score: {silhouette_score(X,label)}')

## Silhouette Plot

In [ ]:
skplt.metrics.plot_silhouette(X, label)
plt.show();

## Grid Search, Rescaling, Data Reduction via PCA

You could do a simple grid search over number of clusters to find the the number of clusters with the "best" metric or score.  You could also try _preprocessing_ your data before clustering.  You could standardize your variables, or you could use PCA to define a reduced space into which they are projected.

## Example 8 Cluster Solution: MiniBatchKMeans

This is very similar to the above.

In [ ]:
kmMB8 = MiniBatchKMeans(n_clusters=8, random_state=11,batch_size=10).fit(X)
labelMB8=kmMB8.predict(X)
pd.Series(labelMB8).value_counts()

In [ ]:
print(f'CH Score: {calinski_harabasz_score(X,labelMB8)}')
print(f'DB Score: {davies_bouldin_score(X,labelMB8)}')
print(f'Sihouette Score: {silhouette_score(X,labelMB8)}')

In [ ]:
skplt.metrics.plot_silhouette(X, labelMB8,figsize=(7,7))
plt.show();

## Graphical Method for Selecting Number of Clusters

Last but not least, here's an example of using a method in _scikit-plot_

In [ ]:
kmMB=MiniBatchKMeans(random_state=88)
skplt.cluster.plot_elbow_curve(kmMB,X,n_jobs=-1,cluster_ranges=range(1,20),
                            figsize=(7,7))
plt.show();